# 🌦 Анализ погоды в Москве, Санкт-Петербурге и Самаре
### Автоматизированная система сравнения метеопараметров

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'colab'

# Стилизация вывода
from IPython.display import display, HTML
display(HTML('''
<style>
.header {
    color: #3498db;
    font-family: Arial;
    text-align: center;
}
.plot-container {
    margin: 20px 0;
    border: 1px solid #eee;
    border-radius: 5px;
    padding: 10px;
}
</style>
'''))

## 📈 Основные метеопараметры

In [ ]:
# Создаем интерактивную панель с 4 графиками
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Температура", "Влажность", "Атм. давление", "Осадки")
)

# Добавляем графики
for city in weather['city'].unique():
    city_data = weather[weather['city'] == city]
    
    fig.add_trace(
        go.Scatter(x=city_data['datetime'], y=city_data['temp_c'], name=f"{city} - Температура"),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=city_data['datetime'], y=city_data['humidity'], name=f"{city} - Влажность"),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Scatter(x=city_data['datetime'], y=city_data['pressure'], name=f"{city} - Давление"),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Bar(x=city_data['datetime'], y=city_data['rain_mm'], name=f"{city} - Осадки"),
        row=2, col=2
    )

# Настройка оформления
fig.update_layout(
    height=800,
    title_text="<b>Сравнение основных метеопараметров</b>",
    hovermode="x unified",
    template="plotly_white"
)

fig.show()

## 🌡️ Тепловая карта температур

In [ ]:
fig = px.imshow(
    pd.pivot_table(weather, values='temp_c', index='city', columns='datetime'),
    labels=dict(x="Дата", y="Город", color="Температура"),
    color_continuous_scale='RdBu_r',
    title='<b>Тепловая карта температур по дням</b>'
)
fig.update_xaxes(side="top")
fig.show()

## 🌀 Ветровые условия

In [ ]:
fig = px.scatter_polar(
    weather, 
    r="wind_speed_kmh", 
    theta="datetime",
    color="city", 
    symbol="city",
    title="<b>Роза ветров по городам</b>",
    template="plotly_dark"
)
fig.update_traces(
    marker=dict(size=8, line=dict(width=1, color='DarkSlateGrey'))
)
fig.show()

## 📊 Сравнительная статистика

In [ ]:
# Создаем интерактивную таблицу с показателями
stats = weather.groupby('city').agg({
    'temp_c': ['mean', 'max', 'min'],
    'humidity': 'mean',
    'wind_speed_kmh': 'max',
    'rain_mm': 'sum'
}).round(1)

fig = go.Figure(data=[go.Table(
    header=dict(
        values=['Город', 'Ср. темп.', 'Макс. темп.', 'Мин. темп.', 'Влажность', 'Макс. ветер', 'Осадки'],
        fill_color='#3498db',
        font=dict(color='white', size=12)
    ),
    cells=dict(
        values=[stats.index] + [stats[col] for col in stats.columns],
        fill_color='lavender',
        align='center'
    ))
])

fig.update_layout(
    title='<b>Сравнительная статистика по городам</b>',
    margin=dict(l=0, r=0, b=0)
)

fig.show()

## 🔍 Выводы

### Температурный режим:
- Самара была самым теплым городом со средней температурой 13.4°C
- Санкт-Петербург оказался самым холодным со средней температурой 4.1°C

### Осадки:
- Санкт-Петербург получил максимальное количество осадков (13.81 мм), включая снег
- Самара оставалась практически без осадков (0 мм)

### Ветровые условия:
- Москва испытала самые сильные порывы ветра до 31.4 км/ч
- В Самаре ветер был более умеренным

### Особенности:
- В Санкт-Петербурге наблюдались снегопады, что необычно для мая
- Москва показала наибольшие суточные перепады температуры
- Самара демонстрировала наиболее стабильные погодные условия